# **Geo-location of German Tweets**

In [ ]:
import numpy as np
import re
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error # use this function to compute MSE, MAE
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
import heapq
import gc
import pickle
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
delimiters = "\n", " ", ".", ",", "?", "!", ":", ";", "\"", "(", ")", "#", "*", "{", "}", "[", "]", "_", "/", "<", ">"
regexPattern = '|'.join(map(re.escape, delimiters))
number_features_BOW = 2700
number_training_samples = 22583
feature_extractor = "BOW"#can be either "BOW" or "W2V_1" #or "W2V_2" or "W2V_3"
truncate_predicted_scores = False
remove_stopwords = True
stemming = True

In [ ]:
def Split_in_parts(row):

  first_part = ""

  i = 0
  while row[i]!='\"':
    first_part = first_part + row[i]
    i+=1
  ind_second_part = i+1

  second_part = row[ind_second_part:-1]#remove the last '\"'


  first_part = first_part[:-1]#remove the last character from first_part

  first_part = first_part.split(",")

  test_row = False
  if len(first_part)!=3:
    test_row = True

  first_part = [float(first_part[i]) for i in range(len(first_part))]#convert first_part from strings to numbers

  tweet_ID = int(first_part[0])

  if test_row==False:
    label  = [first_part[1], first_part[2] ]
    return tweet_ID, label, second_part
  #else
  return tweet_ID, second_part

In [ ]:
germanStemmer=SnowballStemmer("german")
#germanStemmer.stem("suche")
def transform_word(word):
    result = ""
    if remove_stopwords==True:
        if word in stopwords.words('german'):
            result = ""
            return result
    if remove_stopwords==False:
      if word in stopwords.words('german'):
            result = word
            return result
    if stemming==True:
        result = germanStemmer.stem(word)
        return result
    return result
print(transform_word("suche"))
print(stemming)

such
True


# Define feature extractors:



## Feature Extractor 1: Bag of Words

In [ ]:
'''class BagOfWords:
    # a. Initialize the vocabulary in the constructor (it should be empty).
    def __init__(self):
        self.vocab = {
            'UNK': 0
        }
        self.vocab_size = 1

    # b. Implement the 'build_vocabulary' function that takes as input the training essays (numpy array of lists)
    # and build the vocabulary and return the vocabulary size
    def build_dictionary(self, essays):
        for essay in essays:
            for word in essay:
                if word not in self.vocab:
                    self.vocab[word] = self.vocab_size
                    self.vocab_size += 1
        return self.vocab_size

    # c. Implement the 'get_features' function that takes as input the essays and returns the BOW reprezentations
    # (it should be a matrix of NxD, where N is the number of essays and D the length of the dictionary).
    def get_features(self, data):
        results = np.zeros((len(data), self.vocab_size), dtype=np.float32)

        for idx, essay in enumerate(data):
            for word in essay:
                if word not in self.vocab:
                    results[idx, 0] += 1 # for the unknown
                else:
                    results[idx, self.vocab[word]] += 1
       return results
'''


class BagOfWords:
    def __init__(self):
        self.vocab = {
            'UNK': [0,0]#freq, id
        }
        self.vocab_truncated = {
            'UNK': [0]#freq
        }
        self.vocab_size = 1

    def build_dictionary(self, word_lists):
        print("I am in the build_dict...")
        for ind1 in range(len(word_lists)):
            if ind1%1000==0:
              print("ind1 = ", ind1)
            wlist = word_lists[ind1]
            for word in wlist:
                word = transform_word(word)
                if word!="":#this appears sometimes, even when we don't remove the stopwords, so don't change it
                  if word not in self.vocab:
                    self.vocab_size = self.vocab_size + 1
                    self.vocab[word] = [1, self.vocab_size - 1]#freq, id
                  else:
                    self.vocab[word][0]+=1#freq
        #and now:
        print("rebuilding dictionary...")
        self.rebuild_dictionary(number_features_BOW)

        return self.vocab_size

    def rebuild_dictionary(self, number_words):
      for word in self.vocab:
        self.vocab_truncated[word] = self.vocab[word][0]#only freq
      #now truncate to the first number_words words (except first 100 words, which are probably prepositions)
      most_freq = heapq.nlargest(number_words+100,self.vocab_truncated, key=self.vocab_truncated.get)
      print("sunt in functie si am most_freq = ", most_freq)
      #delete the first 100 elements, because they are noise (only in case remove_stopwords==False)
      if remove_stopwords==False:
        most_freq = most_freq[100:]

      print("acum lungimea este", len(most_freq), " iar lista este ", most_freq)

      #free the memory (or try to)
      del self.vocab
      del self.vocab_truncated
      gc.collect()

      #initialize it again
      self.vocab = {
            'UNK': 0#id
        }
      self.vocab_size = 1

      #build a normal dictionary but with less words
      for word in most_freq:
          if word not in self.vocab:
            self.vocab_size = self.vocab_size + 1
            self.vocab[word] = self.vocab_size - 1#id
      print("la sfarsit, avem ", self.vocab_size, " cuvinte")

    def get_features(self, data):
        features = np.zeros((len(data), self.vocab_size), dtype=np.float32)

        for id, tweet in enumerate(data):
            for word in tweet:
                if word not in self.vocab:
                    features[id, 0] += 1 # for the unknown
                else:
                    features[id, self.vocab[word]] += 1
        return features

In [ ]:
'''def Extract_features1(training_data):#bag of words
  BOW = BagOfWords()
  print("build dictionary = ", BOW.build_dictionary(training_data))

  ftrs = BOW.get_features(training_data)
  return ftrs
'''


'def Extract_features1(training_data):#bag of words\n  BOW = BagOfWords()\n  print("build dictionary = ", BOW.build_dictionary(training_data))\n\n  ftrs = BOW.get_features(training_data)\n  return ftrs\n'

## Feature Extractor 2: W2V
(it has get_features_1, get_features_2 and get_features_3)

In [ ]:
def read_vector_embeddings():
      if feature_extractor[0]!='W':
        return None
      with open("/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/vectors_w2v.txt") as f:
        content_w2v_vectors = f.readlines()
      print("first vector embedding: ", content_w2v_vectors[0])
      print("second vector embedding: ", content_w2v_vectors[1])
      print("third vector embedding: ", content_w2v_vectors[2])
      return content_w2v_vectors
content_w2v_vectors = read_vector_embeddings()


In [ ]:
print(len(content_w2v_vectors))

TypeError: ignored

In [ ]:
def split_vector_in_parts(line):
  word = ""
  vector = []
  index = 2#skip the furst character, which for some reason is a lowercase b. Skip also the '
  while line[index]!='\'':
    word = word + line[index]
    index+=1
  index+=1#skip the '

  line = line[index:]
  line = line.split()
  line = [float(nb) for nb in line]

  #if word!=transform_word(word):
  #  print("DIFERIT!\nword=", word, "  stem=", transform_word(word))

  return transform_word(word), line


a = content_w2v_vectors[121]

print(split_vector_in_parts(a))

embedded_words = []
vectors = []
for i in range(len(content_w2v_vectors)):
  if i%5000==1:
    print("i = ", i)
    print("embedded words[-1]:", embedded_words[-1])
  w, v = split_vector_in_parts(content_w2v_vectors[i])
  if w not in embedded_words and w!="":
    embedded_words.append(w)
    vectors.append(v)
print(len(embedded_words))
r = 6/0

TypeError: ignored

In [ ]:
import pickle

output = open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/distinct_stemmed_embedded_words.pkl', 'wb')
pickle.dump(embedded_words, output)
output.close()

output = open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/distinct_stemmed_vectors.pkl', 'wb')
pickle.dump(vectors, output)
output.close()


NameError: ignored

In [ ]:
pkl_file = open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/distinct_stemmed_embedded_words.pkl', 'rb')
embs = pickle.load(pkl_file)
pkl_file.close()

embs
len(embs)

In [ ]:
print("len content vectors = ", len(content_w2v_vectors))
print("len content vectors[0] = ", len(content_w2v_vectors[0]))
print("len content vectors[1] = ", len(content_w2v_vectors[1]))
print("len content vectors[2] = ", len(content_w2v_vectors[2]))
print(type(content_w2v_vectors[0]))

TypeError: ignored

In [ ]:
class Word2Vec:
    def __init__(self):
        self.vocab = {
            'UNK': [0,0]#freq, id
        }
        self.vocab_truncated = {
            'UNK': [0]#freq
        }
        self.vocab_size = 1
        self.vector_embeddings = None

    def build_dictionary(self, word_lists):
        for ind1 in range(len(word_lists)):
            wlist = word_lists[ind1]
            for word in wlist:
                if word!="":#don't know why this appears sometimes
                  if word not in self.vocab:
                    self.vocab_size = self.vocab_size + 1
                    self.vocab[word] = [1, self.vocab_size - 1]#freq, id
                  else:
                    self.vocab[word][0]+=1#freq
        #and now:
        self.rebuild_dictionary(number_features_BOW)

        return self.vocab_size

    def rebuild_dictionary(self, number_words):
      for word in self.vocab:
        self.vocab_truncated[word] = self.vocab[word][0]#only freq
      #now truncate to the first number_words words (except first 100 words, which are probably prepositions)
      most_freq = heapq.nlargest(number_words+100,self.vocab_truncated, key=self.vocab_truncated.get)
      print("sunt in functie si am most_freq = ", most_freq)
      #delete the first 100 elements, because they are noise
      most_freq = most_freq[100:]
      print("acum lungimea este", len(most_freq), " iar lista este ", most_freq)

      #free the memory (or try to)
      del self.vocab
      del self.vocab_truncated
      gc.collect()

      #initialize it again
      self.vocab = {
            'UNK': 0#id
        }
      self.vocab_size = 1

      #build a normal dictionary but with less words
      for word in most_freq:
          if word not in self.vocab:
            self.vocab_size = self.vocab_size + 1
            self.vocab[word] = self.vocab_size - 1#id
      print("la sfarsit, avem ", self.vocab_size, " cuvinte")
      #r = t/0

    def get_features_1(self, data):#average the vectors of the words

      #input:
      features = np.zeros((len(data), self.vocab_size), dtype=np.float32)

      for id, tweet in enumerate(data):
        for word in tweet:
          if word in self.vocab:
            t = 1        #we want to have the vector embedding of this word
      return features

#def text2Vec(line):



# Read the data from files:

In [ ]:

with open("/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/Kaggle data/training.txt") as f:
    content_training = f.readlines()
print("len training = ", content_training)
with open("/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/Kaggle data/validation.txt") as f:
    content_validation = f.readlines()
with open("/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/Kaggle data/test.txt") as f:
    content_test = f.readlines()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print("content_test[0] = ", content_test[0])
print("content_test[1] = ", content_test[1])
print("content_test[2] = ", content_test[2])

content_test[0] =  300121,"👩min vibi funktionkert nöd... 👧hesch d'batterie dri gsteckt? 👩ja, aber das isch nöd sgliiche..."

content_test[1] =  302441,"Ich: Also langsam söti scho schlafe Au Ich: Hesch di scho mal gwundered ob all mensche all farve glich gsehnd? @OJ nei machet ned alli glich vld isch mis grüen dis blau und dis rot mis schwarz vld seitmer drum adligi sind blaublüetig will s bluet eigentlich nöd rot isch, oder doch? #allesefragvodeperspektive Hani mi au scho gfröged Mis rechte Aug gsehts scho ned genau glich wie mis lingge hemmer da es paar stoner? #stonerthoughts"

content_test[2] =  300266,"Hez hie ou lüt wo dr ganz tag ine biudschirm müesse starre u dür das ougeweh becho hei? Heiter irgendwelchi entspannigs/entlastigstipps? Am abe eine blaze Ougearzt. Wahrschinlech bruchsch e brüue. Oder wedä eini hesch e nöii korrektur. regumässig uf wiit entfernti pünkt luege, usem fänschter z.b. Vergiss nid z'blinzle, muesch es chli bewusst mache 😊 Eventuel hocksch ou ds nöch/ds wi

In [ ]:
def Read_training(batch_size, batch_start_index, training_data, training_labels, training_IDs):
  training_data = []
  training_labels = []
  training_IDs = []

  #_________________________________________Training data_________________________________________

  print("batch_size = ", batch_size)
  i = 0
  shortest_text = 2000000000
  for index in range(batch_start_index, batch_start_index+batch_size):
    if i%2000==0:
      print("reading step = ", i)
      print("shortest_text = ", shortest_text)
    row = content_training[index]
    row = row.lower()

    if i==0:
      print("first row in training is: ", row)

    tweet_ID, label, text = Split_in_parts(row)


    splitted_text = re.split(regexPattern, text)

    '''if len(splitted_text)<shortest_text:
      shortest_text = len(splitted_text)
    print("splitted text = ", splitted_text)
    for word in splitted_text:
      haveit = (word in embedded_words)
      print("word = ", word, " have it = ", haveit)
    t = 7/0'''
    training_data.append(splitted_text)
    training_labels.append(label)
    training_IDs.append(tweet_ID)

    i+=1


  #print("finished reading!")
  #print("shortest shortest text = ", shortest_text)
  #r = 6/0

  training_data = np.array(training_data, dtype=object)
  training_labels = np.array(training_labels, dtype=np.float32)
  training_IDs = np.array(training_IDs, dtype=int)

  print("finished converting to np. Now choosing feature extractor:")

  if feature_extractor=="BOW":
    print("instatiating BOW object...")
    BOW = BagOfWords()
    print("BOW instantiated!")
    print("build dictionary = ", BOW.build_dictionary(training_data))
    training_ftrs = BOW.get_features(training_data)

    print(BOW.vocab)

    print("training_ftrs shape= ", training_ftrs.shape)
    print("training_ftrs[0] type= ", type(training_ftrs[0]))
    print("training_ftrs[0][0] type= ", type(training_ftrs[0][0]))
    return training_ftrs, BOW, training_labels, training_IDs

  elif feature_extractor=="W2V_1":
    W2V = Word2Vec()
    training_ftrs = W2V.get_features_1(training_data)
    print("I just calllled get_features_1")
    return training_ftrs, W2V, training_labels, training_IDs
  else:
    r = 7/0
  #return training_ftrs, training_labels, training_IDs

#_________________________________________Now work with test and validation_________________________________________


def Read_test_and_validation(MODEL, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs):
  validation_data = []
  validation_labels = []
  validation_IDs = []
  #_________________________________________Validation data_________________________________________
  i = 0
  print("len content validation = ", len(content_validation))
  for index in range(len(content_validation)):
    if index%1000==0:
      print("index validation = ", index)
    row = content_validation[index]
    row = row.lower()
    if i==0:
      print("first row validation = ", row)
    if i==1:
      print("second row validation = ", row)

    tweet_ID, label, text = Split_in_parts(row)



    splitted_text = re.split(regexPattern, text)


    validation_data.append(splitted_text)
    validation_labels.append(label)
    validation_IDs.append(tweet_ID)
    i+=1



  validation_data = np.array(validation_data)
  validation_labels = np.array(validation_labels)
  validation_IDs = np.array(validation_IDs)

  if feature_extractor=="BOW":
    validation_ftrs = MODEL.get_features(validation_data)
    print(MODEL.vocab)
  elif feature_extractor=="W2V_1":
    validation_ftrs = MODEL.get_features_1(validation_data)
    print("I just calllled get_features_1 for validation")



  test_data = []
  test_IDs = []
  #_________________________________________Test data_________________________________________
  i = 0
  print("len content test = ", len(content_test))
  for index in range(len(content_test)):
    if index%1000==0:
      print("index test = ", index)
    row = content_test[index]
    row = row.lower()
    if i==0:
      print("first row test = ", row)
    if i==1:
      print("second row test = ", row)

    tweet_ID, text = Split_in_parts(row)



    splitted_text = re.split(regexPattern, text)


    test_data.append(splitted_text)
    test_IDs.append(tweet_ID)
    i+=1



  test_data = np.array(test_data)
  test_labels = np.array(test_labels)
  test_IDs = np.array(test_IDs)
  if feature_extractor=="BOW":
    test_ftrs = MODEL.get_features(test_data)
  elif feature_extractor=="W2V_1":
    test_ftrs = MODEL.get_features_1(test_data)
  #print(BOW.vocab)


  return test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs




#free the memory
##del content_training
##del content_test
##del content_validation

In [ ]:
validation_data = []
validation_labels = []
validation_IDs = []

test_data = []
test_labels = []
test_IDs = []

training_data = []
training_labels = []
training_IDs = []

print("len content_training = ", len(content_training))

training_ftrs, bow, training_labels, training_IDs = Read_training(number_training_samples, 0, training_data, training_labels, training_IDs)

print("features type = ", type(training_ftrs))
print("features shape = ", training_ftrs.shape)

most_freq = heapq.nsmallest(200,bow.vocab, key=bow.vocab.get)

#bow.rebuild_dictionary(10000)
test_labels = []
test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)

len content_training =  22583
batch_size =  22583
reading step =  0
shortest_text =  2000000000
first row in training is:  119165,51.810067114093954,10.191330935251802,"seit d vase: ""wenn ich kaputt gang, bringt das 1 jahr unglück."" antwortet de spiegel: ""das isch doch gar nüt. wenn ich kaputt gang, bringt das 7 jahr pech!"" dadruf fangt s kondom a lache. s'atomkraftwerk au 😋 uuuh nöd schlecht 👍 seisch anere frau sie isch fett und du hesch dis lebe lang unglück (au wenn i dem fall nüm lang lebsch)"

reading step =  2000
shortest_text =  2000000000
reading step =  4000
shortest_text =  2000000000
reading step =  6000
shortest_text =  2000000000
reading step =  8000
shortest_text =  2000000000
reading step =  10000
shortest_text =  2000000000
reading step =  12000
shortest_text =  2000000000
reading step =  14000
shortest_text =  2000000000
reading step =  16000
shortest_text =  2000000000
reading step =  18000
shortest_text =  2000000000
reading step =  20000
shortest_text =  2000000

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


{'UNK': 0, 'isch': 1, 'de': 2, 'au': 3, 'i': 4, 'ja': 5, 'e': 6, 'en': 7, 'no': 8, 'uf': 9, 'het': 10, 'bi': 11, 'nid': 12, 'ha': 13, '😂': 14, 'scho': 15, 'vo': 16, 'ned': 17, 'd': 18, 'mer': 19, 'han': 20, 'nod': 21, 'hesch': 22, 'bisch': 23, 'z': 24, '@1': 25, '@2': 26, 'eifach': 27, 'a': 28, 'mal': 29, 'guet': 30, 'imm': 31, 'si': 32, 's': 33, '@oj': 34, 'mach': 35, 'u': 36, '-': 37, 'dr': 38, 'grad': 39, 'gsi': 40, 'mi': 41, 'nei': 42, 'di': 43, 'us': 44, 'kei': 45, '@3': 46, 'jo': 47, 'git': 48, '2': 49, 'hani': 50, 'hend': 51, 'mini': 52, 'me': 53, 'chli': 54, 'ah': 55, 'jodel': 56, 'haha': 57, 'wer': 58, 'ganz': 59, 'war': 60, 'min': 61, 'go': 62, '😅': 63, 'schon': 64, 'weiss': 65, 'meh': 66, 'frau': 67, 'esch': 68, 'mol': 69, '🤔': 70, 'chasch': 71, '1': 72, 'tag': 73, 'dank': 74, 'hut': 75, 'ds': 76, 'cha': 77, '@4': 78, 'bim': 79, 'opp': 80, 'num': 81, 'wieso': 82, 'sich': 83, 'gern': 84, 'glaub': 85, 'nut': 86, 'alt': 87, '😉': 88, 'lieb': 89, 'lut': 90, 'ide': 91, 'eh': 92, '

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# **Normalize the data:**

In [ ]:
def Normalize_data(training_ftrs, validation_ftrs, test_ftrs):
  scaler = preprocessing.Normalizer(norm='l2')
  scaler.fit(training_ftrs)

  # scaling the training data
  scaled_data_train = scaler.transform(training_ftrs)

  # scaling the validation data
  scaled_data_validation = scaler.transform(validation_ftrs)

  # scaling the test data
  scaled_data_test = scaler.transform(test_ftrs)
  #print(scaled_data_test)

  print(np.max(scaled_data_train))
  print(np.mean(scaled_data_train))
  print(np.mean(scaled_data_validation))
  print("shape scaled data train", scaled_data_train.shape)

  return scaled_data_train, scaled_data_validation, scaled_data_test

# **Train the models:**

Train first batch:

In [ ]:


# define the model
#model_krr_x = Ridge() #alpha=10 ** -4, kernel='linear')
#model_krr_y = Ridge()
'''
model_krr_x = KernelRidge(alpha=10 ** -4, kernel='rbf')
model_krr_y = KernelRidge(alpha=10 ** -4, kernel='rbf')
model_SVR_x = SVR(kernel='rbf')
model_SVR_y = SVR(kernel='rbf')

train_X_scores = [line[0] for line in training_labels]
train_Y_scores = [line[1] for line in training_labels]

# train the models
print("training X phase...")
model_krr_x.fit(scaled_data_train, train_X_scores)
###model_SVR_x.fit(scaled_data_train, train_X_scores)
print("training Y phase...")
model_krr_y.fit(scaled_data_train, train_Y_scores)
###model_SVR_y.fit(scaled_data_train, train_Y_scores)
print("training done!")


'''

'''
# define the model
model_krr_x = KernelRidge(alpha=10 ** -4, kernel='rbf')

# train the model
model_krr_x.fit(scaled_data_train, train_scores)

# predict
predicted_labels = kernel_ridge_regression.predict(scaled_data_test)
print(mean_squared_error(predicted_labels, test_scores))
print(mean_absolute_error(predicted_labels, test_scores))

'''

"\n# define the model \nmodel_krr_x = KernelRidge(alpha=10 ** -4, kernel='rbf')\n\n# train the model\nmodel_krr_x.fit(scaled_data_train, train_scores)\n\n# predict\npredicted_labels = kernel_ridge_regression.predict(scaled_data_test)\nprint(mean_squared_error(predicted_labels, test_scores))\nprint(mean_absolute_error(predicted_labels, test_scores))\n\n"

In [ ]:
def Train(model_X, model_Y, train_ftrs_scaled, train_labels):
  #this function will return the trained model

  train_X_scores = [line[0] for line in train_labels]
  train_Y_scores = [line[1] for line in train_labels]

  # train the models
  print("training X phase...")
  model_X.fit(train_ftrs_scaled, train_X_scores)
  ###model_SVR_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_Y.fit(train_ftrs_scaled, train_Y_scores)
  ###model_SVR_y.fit(scaled_data_train, train_Y_scores)
  print("training done!")

  return model_X, model_Y

In [ ]:
def compute_score(true_labels, predicted_labels):
  true_X_labels = np.array([line[0] for line in true_labels], dtype=np.float32)
  true_Y_labels = np.array([line[1] for line in true_labels], dtype=np.float32)

  pred_X_labels = np.array([line[0] for line in predicted_labels], dtype=np.float32)
  pred_Y_labels = np.array([line[1] for line in predicted_labels], dtype=np.float32)


  sum_X = (abs(pred_X_labels - true_X_labels)).mean()
  sum_Y = (abs(pred_Y_labels - true_Y_labels)).mean()


  return sum_X + sum_Y


In [ ]:
listamea = [1,2,3,100]
with open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/numelelelele.pickle', 'wb+') as numepkl:
    pickle.dump(listamea, numepkl, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/numelelelele.pickle', 'rb') as numepkl:
    b = pickle.load(numepkl)

print(b)

[1, 2, 3, 100]


# **Print the distinct GPS points:**

## Only the training labels:

In [ ]:
distinct_training_labels = []
print("type training labels = ", type(training_labels))
print("len training labels = ", len(training_labels))

for label in training_labels:
  found = False
  label = [label[0], label[1]]
  if label not in distinct_training_labels:
      distinct_training_labels.append(label)
print("len distinct_training_labels = ", len(distinct_training_labels))

type training labels =  <class 'numpy.ndarray'>
len training labels =  22583
len distinct_training_labels =  236


## Only validation labels:

In [ ]:
distinct_validation_labels = []
print("type validation labels = ", type(validation_labels))
print("len validation labels = ", len(validation_labels))

for label in validation_labels:
  found = False
  label = [label[0], label[1]]
  if label not in distinct_validation_labels:
      distinct_validation_labels.append(label)
print("len distinctvalidation_labels = ", len(distinct_validation_labels))
print("validation labels:\n", distinct_validation_labels)

type validation labels =  <class 'numpy.ndarray'>
len validation labels =  3044
len distinctvalidation_labels =  358
validation labels:
 [[52.098389261744956, 10.3544964028777], [51.629865771812064, 7.723453237410073], [52.242550335570456, 8.967589928057553], [51.70194630872484, 8.416906474820145], [52.56691275167782, 9.94658273381295], [50.080134228187916, 7.947805755395681], [52.89127516778524, 11.149928057553957], [52.170469798657706, 10.476870503597123], [52.71107382550335, 9.987374100719425], [49.46744966442952, 7.988597122302156], [51.810067114093954, 9.803812949640289], [52.56691275167782, 9.803812949640289], [50.33241610738253, 9.314316546762594], [50.29637583892617, 12.006546762589927], [52.85523489932885, 8.04978417266187], [52.31463087248321, 9.701834532374104], [52.242550335570456, 11.802589928057554], [51.4496644295302, 9.477482014388489], [51.01718120805367, 9.518273381294964], [53.35979865771813, 9.966978417266187], [52.6389932885906, 10.41568345323741], [51.629865771812

## Overall distinct labels:

In [ ]:
distinct_labels_overall = distinct_training_labels
for lbl in distinct_validation_labels:
  if lbl not in distinct_training_labels:
      distinct_labels_overall.append(lbl)
print("len distinct_labels_overall = ", len(distinct_labels_overall))
print(236 + 358)

sorted_x = sorted(distinct_labels_overall, key=lambda x: x[0])
print("____________First 10 x values:____________\n", sorted_x[:10])
print("____________Last 10 x values:____________\n", sorted_x[-10:], "\n\n")
del sorted_x
sorted_y = sorted(distinct_labels_overall, key=lambda x: x[1])
print("____________First 10 y values:____________\n", sorted_y[:10])
print("____________Last 10 y values:____________\n", sorted_y[-10:])
del sorted_y

print("\n\n\t\t\t Distinct xs and ys overall now:\n\n\n")
distinct_xs = []
distinct_ys = []
for lbl in distinct_labels_overall:
  if lbl[0] not in distinct_xs:
    distinct_xs.append(lbl[0])
  if lbl[1] not in distinct_ys:
    distinct_ys.append(lbl[1])
print("len distinct xs = ", len(distinct_xs))
print(distinct_xs)
print("len distinct ys = ", len(distinct_ys))
print(distinct_ys)

print("\n\n\n\n")

print("\n\n\t\t\t Distinct xs and ys for training now:\n\n\n")
distinct_xs_tr = []
distinct_ys_tr = []
for lbl in distinct_training_labels:
  if lbl[0] not in distinct_xs_tr:
    distinct_xs_tr.append(lbl[0])
  if lbl[1] not in distinct_ys_tr:
    distinct_ys_tr.append(lbl[1])
print("len distinct xs_tr = ", len(distinct_xs_tr))
print(distinct_xs_tr)
print("len distinct ys_tr = ", len(distinct_ys_tr))
print(distinct_ys_tr)

print("\n\n\n\n")

print("\n\n\t\t\t Distinct xs and ys for validation now:\n\n\n")
distinct_xs_val = []
distinct_ys_val = []
for lbl in distinct_validation_labels:
  if lbl[0] not in distinct_xs_val:
    distinct_xs_val.append(lbl[0])
    if lbl[0] not in distinct_xs_tr:
      r = 5/0
  if lbl[1] not in distinct_ys_val:
    distinct_ys_val.append(lbl[1])
    if lbl[1] not in distinct_ys_tr:
      r = 5/0

print("len distinct xs_val = ", len(distinct_xs_val))
print(distinct_xs_val)
print("len distinct ys_val = ", len(distinct_ys_val))
print(distinct_ys_val)

print("\n\n\n\n")

len distinct_labels_overall =  594
594
____________First 10 x values:____________
 [[48.17, 8.426023], [48.17, 7.6743183], [48.16999999999999, 8.682050359712232], [48.24356, 8.640796], [48.24356, 8.254205], [48.27812080536913, 8.641258992805753], [48.27812080536913, 7.92741007194245], [48.280342, 8.662272], [48.280342, 7.910568], [48.35020134228188, 8.478093525179858]]
____________Last 10 x values:____________
 [[53.32375838926171, 10.41568345323741], [53.356094, 10.0368185], [53.356094, 10.058295], [53.356094, 9.822045], [53.356095890410955, 10.036818181818184], [53.35979865771813, 9.966978417266187], [53.35979865771813, 10.170935251798564], [53.35979865771813, 10.191330935251802], [53.53999999999999, 9.865000000000002], [53.54, 9.714659]] 


____________First 10 y values:____________
 [[50.41363, 7.03], [51.25959, 7.0514774], [51.29637, 7.0514774], [49.93548, 7.0514774], [50.152214765100666, 7.1115827338129485], [51.26946308724831, 7.13197841726619], [50.54865771812081, 7.13197841726

In [ ]:
def truncate_scores(x_scores, y_scores):
  trunc_x_scores = []
  i1 = 0
  for x in x_scores:
    i1+=1
    dist_min = +100000
    #find the closest truncated value
    closest = -1
    for trunc_x in distinct_xs:
      if abs(trunc_x - x) < dist_min:
        dist_min = abs(trunc_x - x)
        closest = trunc_x
    trunc_x_scores.append(closest)

  trunc_y_scores = []
  i2 = 0
  for y in y_scores:
    i2+=1
    dist_min = +100000
    #find the closest truncated value
    closest = -1
    for trunc_y in distinct_ys:
      if abs(trunc_y - y) < dist_min:
        dist_min = abs(trunc_y - y)
        closest = trunc_y
    trunc_y_scores.append(closest)
  print("i1 = ", i1)
  print("i2 = ", i2)
  return trunc_x_scores, trunc_y_scores

In [ ]:
def convert_to_printable_string(x_scores, y_scores):
  result = "id,lat,long\n"
  for i in range(len(test_IDs)):
    id = test_IDs[i]
    result = result + str(id) + "," + str(x_scores[i]) + "," + str(y_scores[i]) + "\n"
  return result


# Call the previous functions:

In [ ]:
#______________________Training the batch___________________________
training_ftrs, bow, training_labels, training_IDs = Read_training(number_training_samples, 0, training_data, training_labels, training_IDs)
#training_ftrs, bow, training_labels, training_IDs = Read_training(len(content_training), 0, training_data, training_labels, training_IDs)
test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)
#test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs

batch_size =  22583
reading step =  0
shortest_text =  2000000000
first row in training is:  119165,51.810067114093954,10.191330935251802,"seit d vase: ""wenn ich kaputt gang, bringt das 1 jahr unglück."" antwortet de spiegel: ""das isch doch gar nüt. wenn ich kaputt gang, bringt das 7 jahr pech!"" dadruf fangt s kondom a lache. s'atomkraftwerk au 😋 uuuh nöd schlecht 👍 seisch anere frau sie isch fett und du hesch dis lebe lang unglück (au wenn i dem fall nüm lang lebsch)"

reading step =  2000
shortest_text =  2000000000
reading step =  4000
shortest_text =  2000000000
reading step =  6000
shortest_text =  2000000000
reading step =  8000
shortest_text =  2000000000
reading step =  10000
shortest_text =  2000000000
reading step =  12000
shortest_text =  2000000000
reading step =  14000
shortest_text =  2000000000
reading step =  16000
shortest_text =  2000000000
reading step =  18000
shortest_text =  2000000000
reading step =  20000
shortest_text =  2000000000
reading step =  22000
shor

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


{'UNK': 0, 'isch': 1, 'de': 2, 'au': 3, 'i': 4, 'ja': 5, 'e': 6, 'en': 7, 'no': 8, 'uf': 9, 'het': 10, 'bi': 11, 'nid': 12, 'ha': 13, '😂': 14, 'scho': 15, 'vo': 16, 'ned': 17, 'd': 18, 'mer': 19, 'han': 20, 'nod': 21, 'hesch': 22, 'bisch': 23, 'z': 24, '@1': 25, '@2': 26, 'eifach': 27, 'a': 28, 'mal': 29, 'guet': 30, 'imm': 31, 'si': 32, 's': 33, '@oj': 34, 'mach': 35, 'u': 36, '-': 37, 'dr': 38, 'grad': 39, 'gsi': 40, 'mi': 41, 'nei': 42, 'di': 43, 'us': 44, 'kei': 45, '@3': 46, 'jo': 47, 'git': 48, '2': 49, 'hani': 50, 'hend': 51, 'mini': 52, 'me': 53, 'chli': 54, 'ah': 55, 'jodel': 56, 'haha': 57, 'wer': 58, 'ganz': 59, 'war': 60, 'min': 61, 'go': 62, '😅': 63, 'schon': 64, 'weiss': 65, 'meh': 66, 'frau': 67, 'esch': 68, 'mol': 69, '🤔': 70, 'chasch': 71, '1': 72, 'tag': 73, 'dank': 74, 'hut': 75, 'ds': 76, 'cha': 77, '@4': 78, 'bim': 79, 'opp': 80, 'num': 81, 'wieso': 82, 'sich': 83, 'gern': 84, 'glaub': 85, 'nut': 86, 'alt': 87, '😉': 88, 'lieb': 89, 'lut': 90, 'ide': 91, 'eh': 92, '

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## Kernel Ridge Regression:

In [ ]:
#Kernel Ridge Regression

print("__________________________________Work with the first batch__________________________________")

#______________________Training the batch___________________________
training_ftrs, bow, training_labels, training_IDs = Read_training(number_training_samples, 0, training_data, training_labels, training_IDs)
#training_ftrs, bow, training_labels, training_IDs = Read_training(len(content_training), 0, training_data, training_labels, training_IDs)
test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)
#test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs


print(type(training_ftrs[0][0]))
training_ftrs = np.float32(training_ftrs)
print(type(training_ftrs[0][0]))

scaled_data_train, scaled_data_validation, scaled_data_test = Normalize_data(training_ftrs, validation_ftrs, test_ftrs)

#alphas = [10 ** -7, 10 ** -6, 10**-4, 10 ** -3]#, 10 ** -2, 10 ** -1]
alphas = [10 **0, 10 **1, 10 ** -5, 10 ** -2, 10 ** -1]
ps = [8,3]

best_score = 2000000000
best_predicted_labels_1 = []
best_a = -2000000000

for a in alphas:
  print("a currently is ", a)
  model_krr_x = 0
  model_krr_y = 0
  model_krr_x = KernelRidge(alpha=a, kernel='rbf')#, degree = p)
  model_krr_y = KernelRidge(alpha=a, kernel='rbf')#, degree = p)

  ############model_krr_x, model_krr_y = Train(model_krr_x, model_krr_y, scaled_data_train, training_labels)
  train_X_scores = [line[0] for line in training_labels]
  train_Y_scores = [line[1] for line in training_labels]

  # train the models
  print("training X phase...")
  model_krr_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_krr_y.fit(scaled_data_train, train_Y_scores)




  #______________________Predicting for validation__________________________
  print("predincting for X validation...")
  predicted_X_labels_1 = model_krr_x.predict(scaled_data_validation)
  print("predicting for Y validation...")
  predicted_Y_labels_1 = model_krr_y.predict(scaled_data_validation)
  print("predictions done!")

  #_____________________Truncate the scores (if the parameter is true)_________________
  if truncate_predicted_scores == True:
    predicted_X_labels_1, predicted_Y_labels_1 = truncate_scores(predicted_X_labels_1, predicted_Y_labels_1)

  #getting the score
  #predicted_labels_1 = np.array([[predicted_X_labels_1[i], predicted_Y_labels_1[i]] for i in range(len(predicted_X_labels_1))])
  predicted_labels_1 = []
  for i in range(len(predicted_X_labels_1)):
    predicted_labels_1.append([predicted_X_labels_1[i], predicted_Y_labels_1[i]])
  predicted_labels_1 = np.array(predicted_labels_1)
  score = compute_score(predicted_labels_1, validation_labels)

  print("\n\n\ncurrent score is ", score, " and the best score, without this one, was ", best_score, "\n\n\n")


  #______________________Predicting for test__________________________
  print("predincting for X test...")
  predicted_X_labels_1 = model_krr_x.predict(scaled_data_test)
  print("predicting for Y test...")
  predicted_Y_labels_1 = model_krr_y.predict(scaled_data_test)
  print("predictions done!")

  #_____________________Truncate the scores (if the parameter is true)_________________
  if truncate_predicted_scores == True:
    predicted_X_labels_1, predicted_Y_labels_1 = truncate_scores(predicted_X_labels_1, predicted_Y_labels_1)

  nume_fisier = "score=" + str(score) + "krr_a=" + str(a) + "_remove_stopwords=True" + "stemming=" + str(stemming)
  f = open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/Predictions/' + nume_fisier + '.txt', "a")
  string_to_print = convert_to_printable_string(predicted_X_labels_1, predicted_Y_labels_1)
  if score<1.5:
    f.write(string_to_print)
  f.close()

  if score<best_score:
    best_score = score
    best_predicted_labels_1 = predicted_labels_1
    best_a = a

print("best a 1 = ", best_a)

'''La gunoi cu celelalte batch-uri, nu ne trebuie
print("__________________________________Work with the second batch__________________________________")
#______________________Training the batch___________________________
training_ftrs, bow, training_labels, training_IDs = Read_training(11000, 11001, training_data, training_labels, training_IDs)
test_ftrs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)

scaled_data_train, scaled_data_validation = Normalize_data(training_ftrs, validation_ftrs)

alphas = [10 ** -5, 10 ** -4, 10 ** -3, 10 ** -2, 10 ** -1]
ps = [3,2]

best_score = 2000000000
best_predicted_labels_3 = []
best_p = -2000000000

for p in ps:
  a = 10 ** -5
  print("currently p is ", p)
  model_krr_x = 0
  model_krr_y = 0
  model_krr_x = KernelRidge(alpha=a, kernel='polynomial', degree = p)
  model_krr_y = KernelRidge(alpha=a, kernel='polynomial', degree = p)

  ######model_krr_x, model_krr_y = Train(model_krr_x, model_krr_y, scaled_data_train, training_labels)
  train_X_scores = [line[0] for line in training_labels]
  train_Y_scores = [line[1] for line in training_labels]

  # train the models
  print("training X phase...")
  model_krr_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_krr_y.fit(scaled_data_train, train_Y_scores)

  #______________________Predicting#1__________________________
  print("predincting for validation in X...")
  predicted_X_labels_2 = model_krr_x.predict(scaled_data_validation)
  print("predicting for Y...")
  predicted_Y_labels_2 = model_krr_y.predict(scaled_data_validation)
  print("predictions done!")


  #getting the score
  predicted_labels_2 = np.array([[predicted_X_labels_2[i], predicted_Y_labels_2[i]] for i in range(len(predicted_X_labels_2))])
  score = compute_score(predicted_labels_2, validation_labels)

  print("\n\n\ncurrent score is ", score, " and the best score, without this one, was ", best_score, "\n\n\n")

  if score<best_score:
    best_score = score
    best_predicted_labels_2 = predicted_labels_2
    best_p = p
print("best p 2 = ", best_p)

print("__________________________________Work with the third batch__________________________________")

#______________________Training the batch___________________________
training_ftrs, bow, training_labels, training_IDs = Read_training(6500, 2*6500, training_data, training_labels, training_IDs)
test_ftrs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)

scaled_data_train, scaled_data_validation = Normalize_data(training_ftrs, validation_ftrs)

alphas = [10 ** -5, 10 ** -4, 10 ** -3, 10 ** -2, 10 ** -1]
ps = [3,2]

best_score = 2000000000
best_predicted_labels_3 = []
best_p = -2000000000

for p in ps:
  a = 10 ** -5
  print("currently p is ", p)
  model_krr_x = 0
  model_krr_y = 0
  model_krr_x = KernelRidge(alpha=a, kernel='polynomial', degree = p)
  model_krr_y = KernelRidge(alpha=a, kernel='polynomial', degree = p)

  ######model_krr_x, model_krr_y = Train(model_krr_x, model_krr_y, scaled_data_train, training_labels)
  train_X_scores = [line[0] for line in training_labels]
  train_Y_scores = [line[1] for line in training_labels]

  # train the models
  print("training X phase...")
  model_krr_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_krr_y.fit(scaled_data_train, train_Y_scores)

  #______________________Predicting#3__________________________
  print("predincting for X...")
  predicted_X_labels_3 = model_krr_x.predict(scaled_data_validation)
  print("predicting for Y...")
  predicted_Y_labels_3 = model_krr_y.predict(scaled_data_validation)
  print("predictions done!")


  #getting the score
  predicted_labels_3 = np.array([[predicted_X_labels_3[i], predicted_Y_labels_3[i]] for i in range(len(predicted_X_labels_1))])
  score = compute_score(predicted_labels_3, validation_labels)

  print("\n\n\ncurrent score is ", score, " and the best score, without this one, was ", best_score, "\n\n\n")

  if score<best_score:
    best_score = score
    best_predicted_labels_3 = predicted_labels_3
    best_p = p

print("best p 3 = ", best_p)
'''



In [ ]:
3

## K nearest Neighbours Regression:

In [ ]:
#K Nearest Neighbours

print("__________________________________Work with the first batch__________________________________")

#______________________Training the batch___________________________
training_ftrs, bow, training_labels, training_IDs = Read_training(number_training_samples, 0, training_data, training_labels, training_IDs)
#training_ftrs, bow, training_labels, training_IDs = Read_training(len(content_training), 0, training_data, training_labels, training_IDs)
test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)
#test_ftrs, test_IDs, validation_ftrs, validation_labels, validation_IDs


print(type(training_ftrs[0][0]))
training_ftrs = np.float32(training_ftrs)
print(type(training_ftrs[0][0]))

scaled_data_train, scaled_data_validation, scaled_data_test = Normalize_data(training_ftrs, validation_ftrs, test_ftrs)

nghs = [10, 50, 100, 200]

best_score = 2000000000
best_predicted_labels_1 = []
best_ngh = -2000000000

for ngh in nghs:
  print("ngh currently is ", ngh)
  model_knn_x = 0
  model_knn_y = 0
  model_knn_x = KNeighborsRegressor(n_neighbors=ngh, algorithm='kd_tree', weights='distance')
  model_knn_y = KNeighborsRegressor(n_neighbors=ngh, algorithm='kd_tree', weights='distance')

  ############model_krr_x, model_krr_y = Train(model_krr_x, model_krr_y, scaled_data_train, training_labels)
  train_X_scores = [line[0] for line in training_labels]
  train_Y_scores = [line[1] for line in training_labels]

  # train the models
  print("training X phase...")
  model_knn_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_knn_y.fit(scaled_data_train, train_Y_scores)




  #______________________Predicting for validation__________________________
  print("predincting for X validation...")
  predicted_X_labels_1 = model_knn_x.predict(scaled_data_validation)
  print("predicting for Y validation...")
  predicted_Y_labels_1 = model_knn_y.predict(scaled_data_validation)
  print("predictions done!")

  #_____________________Truncate the scores (if the parameter is true)_________________
  if truncate_predicted_scores == True:
    predicted_X_labels_1, predicted_Y_labels_1 = truncate_scores(predicted_X_labels_1, predicted_Y_labels_1)

  #getting the score
  predicted_labels_1 = []
  for i in range(len(predicted_X_labels_1)):
    predicted_labels_1.append([predicted_X_labels_1[i], predicted_Y_labels_1[i]])
  predicted_labels_1 = np.array(predicted_labels_1)
  score = compute_score(predicted_labels_1, validation_labels)

  print("\n\n\ncurrent score is ", score, " and the best score, without this one, was ", best_score, "\n\n\n")


  #______________________Predicting for test__________________________
  print("predincting for X test...")
  predicted_X_labels_1 = model_knn_x.predict(scaled_data_test)
  print("predicting for Y test...")
  predicted_Y_labels_1 = model_knn_y.predict(scaled_data_test)
  print("predictions done!")

  #_____________________Truncate the scores (if the parameter is true)_________________
  if truncate_predicted_scores == True:
    predicted_X_labels_1, predicted_Y_labels_1 = truncate_scores(predicted_X_labels_1, predicted_Y_labels_1)

  nume_fisier = "score=" + str(score) + "knn_ngh=" + str(ngh) + "_remove_stopwords=True" + "stemming=" + str(stemming)
  f = open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/Predictions/' + nume_fisier + '.txt', "a")
  string_to_print = convert_to_printable_string(predicted_X_labels_1, predicted_Y_labels_1)
  if score<1.5:
    f.write(string_to_print)
  f.close()

  if score<best_score:
    best_score = score
    best_predicted_labels_1 = predicted_labels_1
    best_ngh = ngh

print("best ngh = ", best_ngh)

'''

print("__________________________________Work with the second batch__________________________________")
#______________________Training the batch___________________________
training_ftrs, bow, training_labels, training_IDs = Read_training(6500, 6500, training_data, training_labels, training_IDs)
test_ftrs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)

scaled_data_train, scaled_data_validation = Normalize_data(training_ftrs, validation_ftrs)

neighbours = [100, 300, 500]

best_score = 2000000000
best_predicted_labels_1 = []
best_nhg = -2000000000

for nb_ngh in neighbours:
  print("currently nb_ngh is ", nb_ngh)

  model_knn_x = KNeighborsRegressor(n_neighbors=nb_ngh)
  model_knn_y = KNeighborsRegressor(n_neighbors=nb_ngh)

  ############model_krr_x, model_krr_y = Train(model_krr_x, model_krr_y, scaled_data_train, training_labels)
  train_X_scores = [line[0] for line in training_labels]
  train_Y_scores = [line[1] for line in training_labels]

  # train the models
  print("training X phase...")
  model_knn_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_knn_y.fit(scaled_data_train, train_Y_scores)

  #______________________Predicting#1__________________________
  print("predincting for validation in X...")
  predicted_X_labels_2 = model_knn_x.predict(scaled_data_validation)
  print("predicting for Y...")
  predicted_Y_labels_2 = model_knn_y.predict(scaled_data_validation)
  print("predictions done!")


  #getting the score
  predicted_labels_2 = np.array([[predicted_X_labels_2[i], predicted_Y_labels_2[i]] for i in range(len(predicted_X_labels_2))])
  score = compute_score(predicted_labels_2, validation_labels)

  print("\n\n\ncurrent score is ", score, " and the best score, without this one, was ", best_score, "\n\n\n")

  if score<best_score:
    best_score = score
    best_predicted_labels_1 = predicted_labels_1
    best_ngh = nb_ngh

print("best ngh 2 = ", best_ngh)

print("__________________________________Work with the third batch__________________________________")

#______________________Training the batch___________________________
training_ftrs, bow, training_labels, training_IDs = Read_training(6500, 2*6500, training_data, training_labels, training_IDs)
test_ftrs, validation_ftrs, validation_labels, validation_IDs = Read_test_and_validation(bow, validation_data, validation_labels, validation_IDs, test_data, test_labels, test_IDs)

scaled_data_train, scaled_data_validation = Normalize_data(training_ftrs, validation_ftrs)

neighbours = [100, 300, 500]

best_score = 2000000000
best_predicted_labels_1 = []
best_nhg = -2000000000

for nb_ngh in neighbours:
  print("currently nb_ngh is ", nb_ngh)

  model_knn_x = KNeighborsRegressor(n_neighbors=nb_ngh)
  model_knn_y = KNeighborsRegressor(n_neighbors=nb_ngh)

  ############model_krr_x, model_krr_y = Train(model_krr_x, model_krr_y, scaled_data_train, training_labels)
  train_X_scores = [line[0] for line in training_labels]
  train_Y_scores = [line[1] for line in training_labels]

  # train the models
  print("training X phase...")
  model_knn_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_knn_y.fit(scaled_data_train, train_Y_scores)
  #______________________Predicting#3__________________________
  print("predincting for X...")
  predicted_X_labels_3 = model_knn_x.predict(scaled_data_validation)
  print("predicting for Y...")
  predicted_Y_labels_3 = model_knn_y.predict(scaled_data_validation)
  print("predictions done!")


  #getting the score
  predicted_labels_3 = np.array([[predicted_X_labels_3[i], predicted_Y_labels_3[i]] for i in range(len(predicted_X_labels_1))])
  score = compute_score(predicted_labels_3, validation_labels)

  print("\n\n\ncurrent score is ", score, " and the best score, without this one, was ", best_score, "\n\n\n")

  if score<best_score:
    best_score = score
    best_predicted_labels_1 = predicted_labels_1
    best_ngh = nb_ngh

print("best ngh 3 = ", best_ngh)

'''

## Random Forest:

In [ ]:
#Random Forest


print(type(training_ftrs[0][0]))
training_ftrs = np.float32(training_ftrs)
print(type(training_ftrs[0][0]))

scaled_data_train, scaled_data_validation, scaled_data_test = Normalize_data(training_ftrs, validation_ftrs, test_ftrs)

estimators = [500]#also 10?

best_score = 2000000000
best_predicted_labels_1 = []
best_estim = -2000000000

for estim in estimators:
  print("estim currently is ", estim)
  model_rfr_x = 0
  model_rfr_x = RandomForestRegressor(n_estimators=estim, verbose=1, n_jobs=1, warm_start=True)
  model_rfr_y = RandomForestRegressor(n_estimators=estim, verbose=1, n_jobs=1, warm_start=True)#criterion='mae'

  ############model_krr_x, model_krr_y = Train(model_krr_x, model_krr_y, scaled_data_train, training_labels)
  train_X_scores = [line[0] for line in training_labels]
  train_Y_scores = [line[1] for line in training_labels]

  # train the models
  print("training X phase...")
  model_rfr_x.fit(scaled_data_train, train_X_scores)
  print("training Y phase...")
  model_rfr_y.fit(scaled_data_train, train_Y_scores)




  #______________________Predicting for validation__________________________
  print("predincting for X validation...")
  predicted_X_labels_1 = model_rfr_x.predict(scaled_data_validation)
  print("predicting for Y validation...")
  predicted_Y_labels_1 = model_rfr_y.predict(scaled_data_validation)
  print("predictions done!")

  #_____________________Truncate the scores (if the parameter is true)_________________
  if truncate_predicted_scores == True:
    predicted_X_labels_1, predicted_Y_labels_1 = truncate_scores(predicted_X_labels_1, predicted_Y_labels_1)

  #getting the score
  predicted_labels_1 = []
  for i in range(len(predicted_X_labels_1)):
    predicted_labels_1.append([predicted_X_labels_1[i], predicted_Y_labels_1[i]])
  predicted_labels_1 = np.array(predicted_labels_1)
  score = compute_score(predicted_labels_1, validation_labels)

  print("\n\n\ncurrent score is ", score, " and the best score, without this one, was ", best_score, "\n\n\n")


  #______________________Predicting for test__________________________
  print("predincting for X test...")
  predicted_X_labels_1 = model_rfr_x.predict(scaled_data_test)
  print("predicting for Y test...")
  predicted_Y_labels_1 = model_rfr_y.predict(scaled_data_test)
  print("predictions done!")

  #_____________________Truncate the scores (if the parameter is true)_________________
  if truncate_predicted_scores == True:
    predicted_X_labels_1, predicted_Y_labels_1 = truncate_scores(predicted_X_labels_1, predicted_Y_labels_1)

  nume_fisier = "score=" + str(score) + "rfr_estim=" + str(estim) + "_remove_stopwords=True" + "stemming=" + str(stemming)
  f = open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/Predictions/' + nume_fisier + '.txt', "a")
  string_to_print = convert_to_printable_string(predicted_X_labels_1, predicted_Y_labels_1)
  if score<1.4:
    f.write(string_to_print)
  f.close()

  if score<best_score:
    best_score = score
    best_predicted_labels_1 = predicted_labels_1
    best_estim = estim

print("best estim = ", best_estim)


<class 'numpy.float32'>
<class 'numpy.float32'>
1.0
0.0006559493
0.00065651315
shape scaled data train (22583, 2801)
estim currently is  500
training X phase...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed: 194.6min finished


training Y phase...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


## BERT:

In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
import pandas

In [ ]:
lensi = []
for i in range(0, len(content_training)):
  id, lbl, tweet = Split_in_parts(content_training[i])
  #print(len(tweet))
  lensi.append(len(tweet))
lensi.sort()
print(lensi)
print(lensi[len(lensi)//2])
for i in range(len(lensi)):
  if lensi[i]>240:
    print("pozitia = ", i)
    break

[38, 41, 42, 42, 42, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57,

In [ ]:
training_labels[20][0]

50.855

# Varianta 1: prezic X si Y separat


In [ ]:
train_data_for_transformer = []

contor_mare = 0
for i in range(len(content_training)):
  id, lbl, tweet = Split_in_parts(content_training[i])
  if len(tweet)>125:
    contor_mare+=1
    train_data_for_transformer.append(  [tweet, training_labels[i][0]] )
print("tweets longer that 100: ", contor_mare)

train_data_for_transformer = pandas.DataFrame(train_data_for_transformer)
train_data_for_transformer.columns = ["text", "labels"]

tweets longer that 100:  17663


In [ ]:
validation_data_for_transformer = []
validation_to_predict = []
for i in range(len(content_validation)):
  id, lbl, tweet = Split_in_parts(content_validation[i])
  contor_mare+=1
  validation_data_for_transformer.append(  [tweet, validation_labels[i][0]] )
  validation_to_predict.append(tweet)

print("len validation = ", len(validation_data_for_transformer))
validation_data_for_transformer = pandas.DataFrame(validation_data_for_transformer)
validation_data_for_transformer.columns = ["text", "labels"]

len validation =  3044


Definesc modelul:


In [ ]:
nb_epochs = 6

model_args = ClassificationArgs()
model_args.num_train_epochs = nb_epochs
model_args.regression = True
model_args.learning_rate = 9e-6
model_args.overwrite_output_dir=True
model_args.save_model_every_epoch=True
model_args.evaluate_during_training=True
model_args.eval_df=validation_data_for_transformer
model_args.output_dir="/content/outputs"

In [ ]:


model_bert = ClassificationModel(
    "bert",
    "distilbert-base-german-cased",
    num_labels=1,
    args=model_args,

)

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing BertForSequenceClassification: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.lay

In [ ]:
model_bert.train_model(train_data_for_transformer, eval_df=validation_data_for_transformer)

  0%|          | 0/17663 [00:00<?, ?it/s]

Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 0 of 6:   0%|          | 0/2208 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 6:   0%|          | 0/2208 [00:00<?, ?it/s]

In [ ]:
model_bert.train_model(train_data_for_transformer, eval_df=validation_data_for_transformer)

NameError: ignored

In [ ]:
model = ClassificationModel("bert",
    "distilbert-base-german-cased",
    '''args=model_args''')

TypeError: ignored

In [ ]:
result, model_outputs, wrong_predictions = model_bert.eval_model(validation_data_for_transformer)

  0%|          | 0/3044 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/381 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 0.5301031839112285}

In [ ]:
predictions_val, raw_outputs = model_bert.predict(validation_to_predict)

  0%|          | 0/3044 [00:00<?, ?it/s]

  0%|          | 0/381 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
predictions_val

In [ ]:
def print_one_column(lista):
  str = ""

In [ ]:
#for p in predictions_val:
#  print(p)
suma = 0
for i in range(len(predictions_val)):
  dif = abs(predictions_val[i] - validation_labels[i][0])
  suma+=dif
suma = suma/len(predictions_val)
suma
#5.0760718396737134

0.5966592329690101

In [ ]:
str = ""
for

SyntaxError: ignored

In [ ]:
nume_fisier = "bert_x_score=" + str(score)
f = open('/content/drive/MyDrive/Master AI/First Year/Semester 1/PML/Predictions/' + nume_fisier + '.txt', "a")
string_to_print = convert_to_printable_string(predicted_X_labels_1, predicted_Y_labels_1)
  if score<1.5:
    f.write(string_to_print)
  f.close()

###Varianta 1: prezic X si Y deodata



In [ ]:
train_data_for_transformer = []

contor_mare = 0
for i in range(len(content_training)):
  id, lbl, tweet = Split_in_parts(content_training[i])
  if len(tweet)>0:
    contor_mare+=1
    train_data_for_transformer.append(  [tweet, [training_labels[i][0], training_labels[i][1]]] )
print("tweets longer that 100: ", contor_mare)

train_data_for_transformer = pandas.DataFrame(train_data_for_transformer)
train_data_for_transformer.columns = ["text", "labels"]

NameError: ignored

# **Validate the models:**

# Compute the obtained score:
Using the

In [ ]:
#De rulat din nou dupa ce da eroare:


#best_predicted_labels_1 = np.array([[best_predicted_X_labels_1[i], best_predicted_Y_labels_1[i]] for i in range(len(predicted_X_labels_1))])
print("score model 1: ", compute_score(best_predicted_labels_1, validation_labels))

#best_predicted_labels_2 = np.array([[best_predicted_X_labels_2[i], best_predicted_Y_labels_2[i]] for i in range(len(predicted_X_labels_2))])
print("score model 2: ", compute_score(best_predicted_labels_2, validation_labels))

#best_predicted_labels_3 = np.array([[best_predicted_X_labels_3[i], best_predicted_Y_labels_3[i]] for i in range(len(predicted_X_labels_3))])
print("best_score model 3: ", compute_score(best_predicted_labels_3, validation_labels))



avg_prediction = (best_predicted_labels_1 + best_predicted_labels_2 + best_predicted_labels_3) / 3

print("score combined model: ", compute_score(avg_prediction, validation_labels))


